In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import os
import time

In [23]:
main_url="https://townhall.virginia.gov/L/Comments.cfm?GdocForumID=1953"


In [27]:
html_content = requests.get(main_url).text
soup = BeautifulSoup(html_content, "lxml")
main_tab = soup.find('table')

In [73]:
# save off raw html
with open("raw_html.txt", "w") as text_file:
    text_file.write(html_content)

In [ ]:
with open('raw_html.txt', 'r') as file:
    html_content = file.read()

In [28]:
df = pd.read_html(str(main_tab))[0]

In [57]:
urls = [re.search('.+=([0-9]+)', r.a['href']).group(1) for r in main_tab.find_all('tr')]
df['commentId'] = urls[1:]

In [66]:
df.head(3)

,Comment Title,Commenter,Date,commentId,full_comment
0,I am strongly opposed.,Latosha Misciagna,9/26/22 4:27 pm,136285,
1,Awful legislation!,Vincent J. Albiani,9/26/22 4:27 pm,136284,
2,This is a terrible policy decision.,Gaye Degrazio,9/26/22 4:27 pm,136283,


In [41]:
df.tail(3)

,Comment Title,Commenter,Date
7313,Transgender pronkun,Anonymous (128950),9/26/22 5:24 am
7314,Wholly support the 2022 guidance,"LH, Fairfax County Public Schools mother",9/26/22 2:37 am
7315,Protect Kids,Cassidy Moulton-Hill,9/26/22 12:30 am


In [72]:
# loop over all entries, update in place
df['full_comment'] = ''
for i in range(df.shape[0]):
    url=f"https://townhall.virginia.gov/L/viewcomments.cfm?commentid={df.loc[i,'commentId']}"
    soup2 = BeautifulSoup(requests.get(url).text, "lxml")
    df.loc[i,'full_comment'] = soup2.find(attrs={"class": "divComment"}).text

In [75]:
df.to_csv('1953_comments.csv')